# Выбор локации для скважины


**Условия поставленной задачи:**

Нужно решить, где бурить новую скважину.

предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Необходимо построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализировать возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

### **Наблюдения и выводы:**

Для построения моделей и анализа загружены данные по трем регионам. Все датасеты не имеют дубликатов и пропусков. В качестве данных представлены уникальные номера id (которые не будут использоваться для построения моделей), параметры-фичи (f0,f1,f2) и целевой признак -  количество продукта.

Можно отметить для второго региона высокую зависимость параметра f2 и целевого признака product.

**Ход работы:**
- построение отчетов для изучения датасетов;
- разделение выборок на обучающую и валидационную части;
- построение моделей линейной регрессии на обучающей выборке;
- вывод информации о метрике RMSE и среднего запаса предсказанного сырья;
- вывод графика о влиянии фичей на целевой признак;
- расчет прибыли и рисков;
- краткие выводы с рекомендациями по региону для разработки.

In [1]:
#импорт библиотек для работы
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import warnings
from scipy import stats as st

sns.set(style="ticks")

from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.utils import shuffle
from sklearn.linear_model import LinearRegression
from sklearn.compose import make_column_transformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge

!pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip
import pandas_profiling

# настройки
warnings.filterwarnings("ignore")

# константы заглавными буквами
RANDOM_STATE = 12345

BUDGET = 10_000_000_000
COUNT_OF_OIL_WELL = 200
INCOME_ONE = 450_000


     - 17.8 MB 491.5 kB/s 0:00:30m
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.8/395.8 kB 1.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 2.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.0/31.0 MB 3.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━ 11.7/20.8 MB 1.2 MB/s eta 0:00:08^C
     ━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━ 11.8/20.8 MB 1.2 MB/s eta 0:00:08
ERROR: Operation cancelled by user


/var/folders/mq/_xn0hpdj6bb_lwn_8qt3420w0000gp/T/ipykernel_10401/3203091329.py:22: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  import pandas_profiling


In [2]:
#путь к df
path_local = '/Users/evgenij/Desktop/DS/git_portfolio/revenue/'

In [3]:
# импорт датасетов
df_0 = pd.read_csv (path_local + 'geo_data_0.csv')
df_1 = pd.read_csv (path_local + 'geo_data_1.csv')
df_2 = pd.read_csv (path_local + 'geo_data_2.csv')

In [4]:
# выведем краткую информацию о датасетах, сперва идет функция и три поля с выводом информации о каждом датасете
def get_data_info(data):
    display(data.sample(5))
    display(data.info())
    display(data.describe(include='all'))
    display(data.duplicated(['f0', 'f1', 'f2', 'product']).sum())

In [5]:
get_data_info(df_0)

,id,f0,f1,f2,product
18190,3WClx,-0.278483,0.917716,0.784686,10.567672
17904,rvyZb,-0.283333,1.012236,-1.391302,98.376762
17788,wuvKR,0.887400,0.681596,6.658197,61.614535
87127,NyhEx,0.404808,0.951422,1.966316,97.028038
38000,wuf2Z,-0.567908,0.576713,4.463783,155.043724


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

,id,f0,f1,f2,product
count,100000,100000.000000,100000.000000,100000.000000,100000.000000
unique,99990,NaN,NaN,NaN,NaN
top,fiKDv,NaN,NaN,NaN,NaN
freq,2,NaN,NaN,NaN,NaN
mean,NaN,0.500419,0.250143,2.502647,92.500000
std,NaN,0.871832,0.504433,3.248248,44.288691
min,NaN,-1.408605,-0.848218,-12.088328,0.000000
25%,NaN,-0.072580,-0.200881,0.287748,56.497507
50%,NaN,0.502360,0.250252,2.515969,91.849972
75%,NaN,1.073581,0.700646,4.715088,128.564089


0

In [6]:
get_data_info(df_1)

,id,f0,f1,f2,product
7201,b1xHM,-12.515507,-10.510580,2.001330,57.085625
80286,BxjRB,2.828758,-12.375329,1.996655,57.085625
21540,pb31h,4.280399,-8.615953,0.002553,3.179103
2465,xGjQZ,8.312495,-0.415877,4.996855,134.766305
77139,GonQQ,-6.272366,-2.041877,1.003163,30.132364


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

,id,f0,f1,f2,product
count,100000,100000.000000,100000.000000,100000.000000,100000.000000
unique,99996,NaN,NaN,NaN,NaN
top,wt4Uk,NaN,NaN,NaN,NaN
freq,2,NaN,NaN,NaN,NaN
mean,NaN,1.141296,-4.796579,2.494541,68.825000
std,NaN,8.965932,5.119872,1.703572,45.944423
min,NaN,-31.609576,-26.358598,-0.018144,0.000000
25%,NaN,-6.298551,-8.267985,1.000021,26.953261
50%,NaN,1.153055,-4.813172,2.011479,57.085625
75%,NaN,8.621015,-1.332816,3.999904,107.813044


0

In [7]:
get_data_info(df_2)

,id,f0,f1,f2,product
59943,Mf5Oo,-2.925211,-1.616438,1.204724,64.806172
77261,n9s8V,-0.415124,-0.023258,0.810308,134.509775
30994,J1oLs,0.047803,-0.431389,4.067532,10.346698
73680,MrAPx,0.525777,-0.420544,-3.141004,31.469946
593,HhiSc,1.734865,2.060481,1.709261,64.061437


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

,id,f0,f1,f2,product
count,100000,100000.000000,100000.000000,100000.000000,100000.000000
unique,99996,NaN,NaN,NaN,NaN
top,VF7Jo,NaN,NaN,NaN,NaN
freq,2,NaN,NaN,NaN,NaN
mean,NaN,0.002023,-0.002081,2.495128,95.000000
std,NaN,1.732045,1.730417,3.473445,44.749921
min,NaN,-8.760004,-7.084020,-11.970335,0.000000
25%,NaN,-1.162288,-1.174820,0.130359,59.450441
50%,NaN,0.009424,-0.009482,2.484236,94.925613
75%,NaN,1.158535,1.163678,4.858794,130.595027


0

## Обучение и проверка модели

### подготовительная часть

ниже для выполнения задачи сделана функция training_model, которая:

- выделяет фичи и таргет;
- разбивает датасет на тренировочную и валидационную выборки в отношении 75:25;
- выделяет и масштабирует численные признаки;
- удаляет столбец с данными id (для корректного обучения модели);
- выводит график о значимости влияния фичей на таргет;
- обучает модель LinearRegression и делает предсказания по валидационной выборке;
- рассчитывает RMSE (и несколько других метрик) и средний запас предсказанного сырья

на экран выводится следующая информация:

- RMSE модели;
- средний запас предсказанного сырья;
- средний запас сырья по датасету

Функция возвращает два значения - предсказанные ответы (predicted_valid) и правильные ответы (target_valid)


### выводы согласно задаче:

для первого датасета получились такие результаты:

    RMSE: 37.5794217150813
    средний запас предсказанного сырья: 92.59256778438035
    средний запас сырья: 92.50000000000001

для второго датасета:

    RMSE: 0.893099286775617
    средний запас предсказанного сырья: 68.728546895446
    средний запас сырья: 68.82500000000002

для третьего датасета:

    RMSE: 40.02970873393434
    средний запас предсказанного сырья: 94.96504596800489
    средний запас сырья: 95.00000000000004

по итогам обучения модели по разным датасетам самая низкая и лучшая метрика RMSE у модели, обучающейся на первом датасете. При этом средний запас предсказанного сырья все модели оценивают адекватно относительно среднего значения запаса сырья.

дополнительная информация

В части второго датасета df_1 стоит обратить внимание, что ярко выражено влияние одной из фичей (f2), при этом остальные (f0 и f1) не оказывают никакого влияния на таргет. Для остальных датасетов (df_0 и df_2) также наблюдается среднее влияние f2 на таргет, но при этом остальные фичи также имеют влияние.

Также в первой главе проекта при построении отчета наблюдается сильная корреляция для df1 между product (таргет) и f2 (фича).

Рекомендуется при выборе модели учитывать, что модель df_1 имеет лучшие показатели RMSE относительно других (df_0 & df_2).

In [11]:
#функция на вход которой подается датасет, в функции назначаются цели и фичи, разбиваются выборки, проводится масштабирование
#числовых данных и отбрасывается столбец с id номером, обучается модель и делается предсказания на валидационной выборке.
# также функция выдает информацию о влиянии фичей на цель
# рассчитывается RMSE и другие метрики и на выходе функции выдается: предсказания и правильные ответы на валидационной выборке 
# и выводится на экран RMSE, средний запас предсказанного сырья и среднй запас сырья по датасету
def training_model (data, drop_column = 'id', target_column = 'product'):
#назначим фичи и цель
    target = data[target_column]
    features = data.drop(target_column, axis=1)
#разобьем на тренировочную и валидационную выборки (пропорция 75:25)
    features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=RANDOM_STATE)
    
    
#удалим столбцы с id номером    
    features_train = features_train.drop(drop_column, axis=1)
    features_valid = features_valid.drop(drop_column, axis=1)

#выведем численные признаки
    num_features = features_train.select_dtypes(exclude='object').columns.to_list()    
    
#выведем информацию о влиянии фичей на цель
#________начало блока_____________
    ss = StandardScaler()
    X_train_scaled = ss.fit_transform(features_train)
    X_test_scaled = ss.transform(features_valid)
    y_train = np.array(target_train)

    rfc = RandomForestRegressor()
    rfc.fit(X_train_scaled, y_train)
    display(rfc.score(X_train_scaled, target_train))

    feats = {}
    for feature, importance in zip(features_train.columns, rfc.feature_importances_):
        feats[feature] = importance
    importances = pd.DataFrame.from_dict(feats, orient='index').rename(columns={0: 'Gini-Importance'})
    importances = importances.sort_values(by='Gini-Importance', ascending=False)
    importances = importances.reset_index()
    importances = importances.rename(columns={'index': 'Features'})
    sns.set(font_scale = 5)
    sns.set(style="whitegrid", color_codes=True, font_scale = 1.7)
    fig, ax = plt.subplots()
    fig.set_size_inches(30,15)
    sns.barplot(x=importances['Gini-Importance'], y=importances['Features'], data=importances, color='skyblue')
    plt.xlabel('Важность features', fontsize=25, weight = 'bold')
    plt.ylabel('Признаки', fontsize=25, weight = 'bold')
    plt.title('Влияние features на target', fontsize=25, weight = 'bold')
    display(plt.show())
    display(importances)    
#________конец блока_____________
    
# масштабируем численные признаки
    pd.options.mode.chained_assignment = None
    scaler = StandardScaler()
    scaler.fit(features_train[num_features])
    features_train[num_features] = scaler.transform(features_train[num_features])
    features_valid[num_features] = scaler.transform(features_valid[num_features])
    
#обучим модель и сделаем предсказание
    model = LinearRegression()
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    predicted_valid = pd.Series(predicted_valid)
#посчитаем метрики
    result = (mean_squared_error(target_valid, predicted_valid))**0.5
    print("RMSE:", result)
    mae = mean_absolute_error(target_valid, predicted_valid)
    print('MAE:', mae)
    mape = mean_absolute_percentage_error(target_valid, predicted_valid)
    print('MAPE:', mape)
    
    print('средний запас предсказанного сырья:',round(predicted_valid.mean(),3))
    print('средний запас сырья:',round(target.mean(),3))
    

# возвращаем для дальнейшей работы массив предсказанных данных и массив правильных ответов    
    return predicted_valid, target_valid, result

In [12]:
# выведем информацию по обучению модели на первом датасете df_0 и сохраним для него предсказанные ответы и правильные ответы
predicted_valid_0, target_valid_0, result_0 = training_model(df_0)

0.8918648390152065

None

,Features,Gini-Importance
0,f2,0.472152
1,f1,0.269935
2,f0,0.257914


RMSE: 37.5794217150813
MAE: 30.919600777151313
MAPE: 2.1224513911488185
средний запас предсказанного сырья: 92.593
средний запас сырья: 92.5


In [13]:
# аналогичная работа для второго датасета df_1
predicted_valid_1, target_valid_1, result_1 = training_model(df_1)

0.9999635372625467

None

,Features,Gini-Importance
0,f2,0.998877
1,f0,0.001038
2,f1,0.000085


RMSE: 0.8930992867756168
MAE: 0.7187662442124757
MAPE: 270028896017520.3
средний запас предсказанного сырья: 68.729
средний запас сырья: 68.825


In [14]:
# аналогичная работа для второго датасета df_2
predicted_valid_2, target_valid_2, result_2 = training_model(df_2)

0.8922956470486232

None

,Features,Gini-Importance
0,f2,0.431738
1,f1,0.285344
2,f0,0.282918


RMSE: 40.02970873393434
MAE: 32.792652105481814
MAPE: 10315299110670.121
средний запас предсказанного сырья: 94.965
средний запас сырья: 95.0


### **важное наблюдение**

при дальнейшей работе на этапе построения возможной прибыли для датасета по второму региону наблюдается, что модель предсказывает отрицательные значения продукта. Возможно, имеет влияение фичей f0 & f1, которые правда не оказывают влияния на целевой параметр (корреляция почти нулевая). Для теста ниже построена вторая модель по второму региону, из параметров оставлена только f2.

In [15]:
# построим модель для данных по второму региону,оставив только одну фичу
predicted_valid_1_v2, target_valid_1_v2, result_1_v2 = training_model(df_1, drop_column = ['id', 'f0', 'f1'])

0.9996401542197287

None

,Features,Gini-Importance
0,f2,1.0


RMSE: 1.5942275477363055
MAE: 1.5884709258698513
MAPE: 592101945503034.2
средний запас предсказанного сырья: 68.729
средний запас сырья: 68.825


## Подготовка к расчёту прибыли

### Ключевые значения
Все ключевые значения для расчётов сохранены в отдельных переменных: budget, count_of_oil_well & income_one.
 
Из условий задачи:

При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
Бюджет на разработку скважин в регионе — 10 млрд рублей.
При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.



In [16]:
# рассчитаем сколько тыс. баррелей нефти нужно добыть, чтобы отбить в ноль бюджет на разработку скважин в регионе.
# возьмем весь бюджет в 10 млрд рублей, разделим на 450 тыс рублей (доход с 1 тыс бареллей) и разделим на 200 точек дла разработки
# получим сколько должна точка выдавать минимум тыс бареллей нефти, чтобы отбить затраты на разработку скважин 


target_mean_barel = BUDGET / INCOME_ONE / COUNT_OF_OIL_WELL
print('достаточный объём сырья для безубыточной разработки новой скважины:',round(target_mean_barel,3))

достаточный объём сырья для безубыточной разработки новой скважины: 111.111


###  Сравнение полученного объёма сырья со средним запасом в каждом регионе. 

In [17]:
#функция проверяем по условиям задачи убытки в 2,5%
def check_loss (name):
    balance_ = -round(100 - name.mean()*100/target_mean_barel, 2)
    print('относительная разница между предсказанным моделью средним объемом и минимальным безубыточным объемом сырья:',
          balance_)
    if balance_ > 2.5:
        print('вероятность убытков выше 2,5%')
    else:
        print('вероятность убытков ниже 2,5%')
    return balance_

In [18]:
# проверим разницу между предсказанным средним значением сырья по первой модели и безубыточным значением для разработки новой скважины
balance_0 = check_loss(predicted_valid_0)

относительная разница между предсказанным моделью средним объемом и минимальным безубыточным объемом сырья: -16.67
вероятность убытков ниже 2,5%


In [19]:
#аналогично для второй модели
balance_1 = check_loss(predicted_valid_1)

относительная разница между предсказанным моделью средним объемом и минимальным безубыточным объемом сырья: -38.14
вероятность убытков ниже 2,5%


In [20]:
#аналогично для второй модели
balance_1_v2 = check_loss(predicted_valid_1_v2)

относительная разница между предсказанным моделью средним объемом и минимальным безубыточным объемом сырья: -38.14
вероятность убытков ниже 2,5%


In [21]:
#аналогично для третьей модели
balance_2 = check_loss(predicted_valid_2)

относительная разница между предсказанным моделью средним объемом и минимальным безубыточным объемом сырья: -14.53
вероятность убытков ниже 2,5%


### **наблюдения:**

по расчетам минимальным объемом сырья для безубыточной разработки новой скважины рассчитаем в 111,11 тыс. бареллей

если сравнить со средними значениями в каждом регионе: 1 регион - 92,5 тыс. бареллей, 2 регион - 68,7 тыс. бареллей и 3 регион - 94,9 тыс. бараллей, то по условиям задачи (оставить лишь те регионы, в которых вероятность убытков меньше 2.5%)

## Расчёт прибыли и рисков 

### подготовительная часть

ниже представлена функция, которая:
- выбирает скважины с максимальными значениями предсказаний;
- суммирует целевое значение объёма сырья, соответствующее этим предсказаниям;
- рассчитывает прибыль для полученного объёма сырья;
- применяется техника Bootstrap с 1000 выборок, чтобы найти распределение прибыли;

на выходе функции выдается  средняя прибыль, 95%-й доверительный интервал и риск убытков.

In [22]:
def revenue_full (predicted_valid, target_valid):
# сбросим индексы в target_valid
    target_valid = target_valid.reset_index(drop=True)
# заявим state
    state = np.random.RandomState(12345)
# сделаем вложенную функцию, которая подсчитывает и возвращает выручку.  
    def revenue_in(target, probabilities, count):


        probs_sorted = probabilities.sort_values(ascending=False)
        selected = target[probs_sorted.index][:count]
        return INCOME_ONE * selected.sum() - BUDGET
#Применим технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли
# для target_subsample берем n=500 (При разведке региона исследуют 500 точек), при расчете выручки берем 200 лучших точек
    values = []
    for i in range(1000):
        target_subsample = target_valid.sample (n=500, replace=True, random_state=state)
        probs_subsample = predicted_valid[target_subsample.index]
    
        values.append(revenue_in(target_subsample, probs_subsample,COUNT_OF_OIL_WELL))    
    values = pd.Series(values)
# рассчитаем среднюю выручку
    mean = values.mean()
#рассчитаем 95% доверительный интервал через квантили 0.025 (нижний) и 0.975 (верхний)
    lower = round(values.quantile(q = 0.025),2)
    higher = round(values.quantile(q = 0.975),2)
#рассчитаем 95% доверительный интервал через interval
    alpha = 1 - 0.05
    confidence_interval = st.t.interval(alpha=alpha, df = len(values)-1, loc=values.mean(), scale=values.sem())    
    
    
# рассчитаем риск убытков
    risk = ((pd.Series(values)<0).mean())
    print('Риск убытков = {:.2%} '.format(risk))

#вывод результатов    
    print("95%-ый доверительный интервал:", [lower, higher])

    print("Средняя выручка:", round(mean,2))

    return mean, risk


In [23]:
# расчет для первого региона df_0
print ("данные о средней выручке для первого региона:")
mean_0, risk_0 = revenue_full(predicted_valid_0, target_valid_0)

данные о средней выручке для первого региона:
Риск убытков = 6.00% 
95%-ый доверительный интервал: [-102090094.84, 947976353.36]
Средняя выручка: 425938526.91


In [24]:
# расчет для второго региона df_1
print ("данные о средней выручке для второго региона:")
mean_1, risk_1 = revenue_full(predicted_valid_1, target_valid_1)

данные о средней выручке для второго региона:
Риск убытков = 1.00% 
95%-ый доверительный интервал: [68873225.37, 931547591.26]
Средняя выручка: 515222773.44


In [25]:
# расчет для третьего региона df_2
print ("данные о средней выручке для третьего региона:")
mean_2, risk_2 = revenue_full(predicted_valid_2, target_valid_2)

данные о средней выручке для третьего региона:
Риск убытков = 6.40% 
95%-ый доверительный интервал: [-128880547.33, 969706954.18]
Средняя выручка: 435008362.78


In [26]:
#выведем информацию о предсказанных данных для второго региона df_1
# можно заметить, что модель предсказывает отрицательные значения
predicted_valid_1.sort_values(ascending=False)

20430    139.818970
7777     139.773423
8755     139.703330
1178     139.560938
4285     139.516754
            ...    
5085      -1.634717
12426     -1.693844
9852      -1.871442
5617      -1.883737
4975      -1.893774
Length: 25000, dtype: float64

In [27]:
# расчет для второго региона для второй версии модели, из которой исключили вспе фичи, кроме f2
mean_1_v2, risk_1_v2 = revenue_full(predicted_valid_1_v2, target_valid_1_v2)

Риск убытков = 1.70% 
95%-ый доверительный интервал: [49572617.14, 925019718.77]
Средняя выручка: 495457656.86


In [28]:
# напишем функцию для расчета средней добычи
def mean_predict_200 (data):

    return data[:200].sort_values(ascending=False).mean()

In [29]:
mean_predict_200_0 = mean_predict_200(predicted_valid_0)

In [30]:
mean_predict_200_1 = mean_predict_200(predicted_valid_1)

In [31]:
mean_predict_200_1_v2 = mean_predict_200(predicted_valid_1_v2)

In [32]:
mean_predict_200_2 = mean_predict_200(predicted_valid_2)

In [33]:
product_all_0 = round(target_valid_0.sum(),3)

In [34]:
product_all_1 = round(target_valid_1.sum(),3)

In [35]:
product_all_2 = round(target_valid_2.sum(),3)

подготовим таблицы для выводов

In [36]:
summary_table = {'Регион':['первый регион df_0','второй регион df_1','третий регион df_2'],
                'Средний запас предсказанного сырья, тыс. барр':[predicted_valid_0.mean(), predicted_valid_1.mean(), 
                                                     predicted_valid_2.mean()],
                'Средний запас предсказанного сырья (top200), тыс. барр':[mean_predict_200_0, mean_predict_200_1, 
                                                     mean_predict_200_2],
                'Средняя выручка, млрд руб.':[round(mean_0/1000000000,2), round(mean_1/1000000000,2), 
                                             round(mean_2/1000000000,2)],
                'Риск убытка':[risk_0, risk_1, risk_2],
                'дельта, %':[balance_0, balance_1, balance_2],
                'запасы сырья, тыс.барр':[product_all_0, product_all_1, product_all_2],
                'RMSE модели':[result_0, result_1, result_2]}

In [37]:
summary_table_df_2 = {'Регион':['второй регион df_1','второй регион df_1_v2'],
                'Средний запас предсказанного сырья, тыс. барр': [predicted_valid_1.mean(), 
                                                      predicted_valid_1_v2.mean()],
                'Средний запас предсказанного сырья (top200), тыс. барр':[mean_predict_200_1, 
                                                      mean_predict_200_1_v2],
                'Средняя выручка, млрд руб.':[round(mean_1/1000000000,2), 
                                              round(mean_1_v2/1000000000,2)],
                'Риск убытка':[risk_1, risk_1_v2],
                'дельта, %':[balance_1, balance_1_v2],
                'RMSE модели':[result_1, result_1_v2]}

In [38]:
terms = {'Термины и константы':['бюджет', 'количество регионов', 'Доход с каждой единицы продукта',
                                'количество скважин для разведки', 'количество скважин для разработки (top200)',
                               'достаточный объём сырья для безубыточной разработки новой скважины', 'табл.: дельта'],
        'Значения': ['10 млрд руб', '3', '450 тыс. руб. с 1 тыс. барр продукта','500','200','111,11',
                     'относительная разница между предсказанным средним объемом и минимальным безубыточным объемом сырья',]}

In [39]:
summary_table_df_2 = pd.DataFrame(summary_table_df_2)
summary_table = pd.DataFrame(summary_table)
terms = pd.DataFrame(terms)


In [40]:
numeric_columns = ['Средний запас предсказанного сырья, тыс. барр', 'Средний запас предсказанного сырья (top200), тыс. барр',
                   'Средняя выручка, млрд руб.', 'дельта, %', 'запасы сырья, тыс.барр']
rmse_column = ['RMSE модели', 'Риск убытка']
summary_table = (summary_table
                 .style
                 .highlight_max(color='yellowgreen', subset=numeric_columns)
                 .highlight_min(color='coral', subset=numeric_columns)
                 .highlight_max(color='coral', subset=rmse_column)
                 .highlight_min(color='yellowgreen', subset=rmse_column)
)

### **выводы: предложения по региону для разработки скважин и обоснование выбора.**

короткая таблица о терминах, константах и значениях

In [41]:
terms

,Термины и константы,Значения
0,бюджет,10 млрд руб
1,количество регионов,3
2,Доход с каждой единицы продукта,450 тыс. руб. с 1 тыс. барр продукта
3,количество скважин для разведки,500
4,количество скважин для разработки (top200),200
5,достаточный объём сырья для безубыточной разра...,"111,11"
6,табл.: дельта,относительная разница между предсказанным сред...


в рамках задачи было выдвинуто предположение: для модели по второму региону была замечана сильная корреляция цели и фичи f2. Предполагалось проверить, если оставить для данной модели для обучения только один параметр f2, то будет ли сильно влиять на показатели для бизнеса. Однако, анализ показал, что особых изменений не обнаружено, поэтому для дальнейшей таблицы используется данные из модели по второму региона, которая для обучения берет все фичи. Также, оставив только одну фичу во второй версии модели, мы ухудшили показатель RMS, а также увеличили риски убытка

In [42]:
summary_table_df_2

,Регион,"Средний запас предсказанного сырья, тыс. барр","Средний запас предсказанного сырья (top200), тыс. барр","Средняя выручка, млрд руб.",Риск убытка,"дельта, %",RMSE модели
0,второй регион df_1,68.728547,66.729502,0.52,0.010,-38.14,0.893099
1,второй регион df_1_v2,68.728943,66.688852,0.50,0.017,-38.14,1.594228


ниже представлена сравнительная таблица с показателями по трем регионам, зеленым выделены значения с лучшими показателями, желтым с худшими показателями

In [43]:
summary_table

,Регион,"Средний запас предсказанного сырья, тыс. барр","Средний запас предсказанного сырья (top200), тыс. барр","Средняя выручка, млрд руб.",Риск убытка,"дельта, %","запасы сырья, тыс.барр",RMSE модели
0,первый регион df_0,92.592568,91.301957,0.430000,0.060000,-16.670000,2301964.919000,37.579422
1,второй регион df_1,68.728547,66.729502,0.520000,0.010000,-38.140000,1718078.401000,0.893099
2,третий регион df_2,94.965046,98.221002,0.440000,0.064000,-14.530000,2372105.820000,40.029709


### **выводы: рекомендации по выбору региона для разработки:**

Исходя из условий задачи, что выбирается регион с наилучшей средней выручкой - предлагается для разработки второй регион (средняя выручка 10,5 млрд рублей, оцениваемая прибыть 0,52 млрд рублей, средний запас предсказанного сырья в одной точке 68,7 тыс. бареллей, запасы сырья порядка 1 718 078 тыс. бареллей, RMSE модели 0,89). **Риск убытка составляет 1%**.